In [2]:
!pip install unidecode

     |████████████████████████████████| 245kB 2.7MB/s 


In [1]:
!pip install category_encoders

     |████████████████████████████████| 81kB 2.2MB/s 


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
import xgboost as xgb
import re
import unidecode
import nltk
from nltk.tokenize import TweetTokenizer
import string
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
import category_encoders as ce

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
id='1cQhZ_2NeVAXqkXpmaCQbJVxLP4b1lNAf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

id='12nWwwnj9jbtAsk0VENlQaV3d6jclQ-1S'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [7]:
def remove_punctuation(word):        
    clean_word = ''.join([char for char in word if char not in string.punctuation])
    return clean_word

def cleaning_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    text_tokenize = tokenizer.tokenize(text)
    wordlist = []
    for word in text_tokenize:
        word = word.lower()
        word = re.sub('(?P<url>https?://[^\s]+)', ' ', word)
        word = remove_punctuation(word)
        word = re.sub(r'[^\w]', ' ', word)
        word = unidecode.unidecode(word)
        word = re.sub(r'[0-9]','', word)
        if((word != '')&(word != ' ')&(word not in stopwords)):
            wordlist.append(word)
    clean_text = ' '.join(wordlist)
    return clean_text

In [8]:
def concatenate(x,char):
    words = ""
    for word in x:
        if word.startswith(char):
            words = words + word + " "
    return words

def count_vowels(x):
    return (x.count('a') + x.count('e') + x.count('i') + x.count('o') + x.count('u'))

def count_short_words(x):
    count = 0
    words = x.split(' ')
    for word in words:
        if 1 <= len(word) <= 3:
            count += 1
    return count

def count_stopwords(x):
    count = 0
    words = x.split(' ')
    for word in words:
        if word in stopwords:
            count += 1
    return count

In [9]:
train["special_chars_count"] =  train["text"]
train["special_chars_count"] =  train["special_chars_count"].str.lower()
train["special_chars_count"] = train["special_chars_count"].apply(lambda x: re.sub(r'[a-z]','',x))
train["special_chars_count"] = train["special_chars_count"].str.strip()
train["special_chars_count"] = train["special_chars_count"].apply(lambda x: re.sub(' +','', x))
train["special_chars_count"] = train["special_chars_count"].apply(lambda x: re.sub(r'[0-9]','', x))
train["special_chars_count"] = train["special_chars_count"].str.len()

train["hashtags"] = train["text"].str.lower().str.split(' ').apply(lambda x: concatenate(x,'#'))
train["labels"] = train["text"].str.lower().str.split(' ').apply(lambda x: concatenate(x,'@'))
train["hashtags_count"] = train["hashtags"].str.split(' ').apply(lambda x: len(x))-1
train["labels_count"] = train["labels"].str.split(' ').apply(lambda x: len(x))-1

train["num_chars_count"] = train["text"]
train["num_chars_count"] =  train["num_chars_count"].str.lower()
train["num_chars_count"] = train["num_chars_count"].apply(lambda x: re.sub(r'[a-z]','',x))
train["num_chars_count"] = train["num_chars_count"].apply(lambda x: re.sub(r'[^\w]','',x))
train["num_chars_count"] = train["num_chars_count"].str.strip()
train["num_chars_count"] = train["num_chars_count"].str.len()

train["links_count"] = train['text'].apply(lambda x: len([w for w in str(x).lower().split()
                                                           if 'http' in w or 'https' in w]))

train["clean_text"] = train["text"].apply(lambda x: cleaning_text(x)) # para el bag of words

train["text"] = train["text"].str.lower()
train["text"] = train["text"].apply(lambda x: re.sub('(?P<url>https?://[^\s]+)', ' ', x))
train["text"] = train["text"].apply(lambda x: re.sub(r'[^\w]', ' ', x))
train["text"] = train["text"].apply(lambda x: re.sub(r'_', ' ', x))
train["text"] = train["text"].apply(lambda x: re.sub(r'[0-9]',' ', x))
train["text"] = train["text"].apply(lambda x: re.sub(' +',' ', x))
train["text"] = train["text"].apply(lambda x: unidecode.unidecode(x))
train["text"] = train["text"].str.strip()
train["text_length"] = train["text"].str.len()

train["mean_word_length"] = train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
train["vowels_count"] = train["text"].apply(lambda x: count_vowels(x))
train["short_words_count"] = train["text"].apply(lambda x: count_short_words(x))
train["stopwords_count"] = train["text"].apply(lambda x: count_stopwords(x))
train["text"] = train["text"].apply(lambda x: re.sub(r'\b\w{1}\b', '', x))
train["words_count"] = train["text"].str.split(' ').apply(lambda x: len(x))

train["keyword"] = train["keyword"].str.replace('%20',' ')
train["keyword"] = train["keyword"].astype('category')

train.rename(columns={"target":"target_label"}, inplace=True)
train.rename(columns={"location":"location_original"}, inplace=True)
train.rename(columns={"id":"id_original"}, inplace=True)
train.rename(columns={"text":"text_original"}, inplace=True)
train.rename(columns={"keyword":"keyword_original"}, inplace=True)

train.head()

,id_original,keyword_original,location_original,text_original,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,1,#earthquake,,1,0,0,0,deeds reason earthquake may allah forgive us,68,4.307692,25,7,6,13
1,4,NaN,NaN,forest fire near la ronge sask canada,1,1,,,0,0,0,0,forest fire near la ronge sask canada,37,4.428571,13,1,0,7
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,3,,,0,0,0,0,residents asked shelter place notified officer...,130,4.954545,45,9,11,22
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1,2,#wildfires,,1,0,5,0,people receive wildfires evacuation orders cal...,56,7.142857,24,1,1,7
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,2,#alaska #wildfires,,2,0,0,0,got sent photo ruby alaska smoke wildfires pou...,85,4.375000,25,3,7,16


In [10]:
test["special_chars_count"] =  test["text"]
test["special_chars_count"] =  test["special_chars_count"].str.lower()
test["special_chars_count"] = test["special_chars_count"].apply(lambda x: re.sub(r'[a-z]','',x))
test["special_chars_count"] = test["special_chars_count"].str.strip()
test["special_chars_count"] = test["special_chars_count"].apply(lambda x: re.sub(' +','', x))
test["special_chars_count"] = test["special_chars_count"].apply(lambda x: re.sub(r'[0-9]','', x))
test["special_chars_count"] = test["special_chars_count"].str.len()

test["hashtags"] = test["text"].str.lower().str.split(' ').apply(lambda x: concatenate(x,'#'))
test["labels"] = test["text"].str.lower().str.split(' ').apply(lambda x: concatenate(x,'@'))
test["hashtags_count"] = test["hashtags"].str.split(' ').apply(lambda x: len(x))-1
test["labels_count"] = test["labels"].str.split(' ').apply(lambda x: len(x))-1

test["num_chars_count"] = test["text"]
test["num_chars_count"] =  test["num_chars_count"].str.lower()
test["num_chars_count"] = test["num_chars_count"].apply(lambda x: re.sub(r'[a-z]','',x))
test["num_chars_count"] = test["num_chars_count"].apply(lambda x: re.sub(r'[^\w]','',x))
test["num_chars_count"] = test["num_chars_count"].str.strip()
test["num_chars_count"] = test["num_chars_count"].str.len()

test["links_count"] = test['text'].apply(lambda x: len([w for w in str(x).lower().split()
                                                           if 'http' in w or 'https' in w]))

test["clean_text"] = test["text"].apply(lambda x: cleaning_text(x))

test["text"] = test["text"].str.lower()
test["text"] = test["text"].apply(lambda x: re.sub('(?P<url>https?://[^\s]+)', ' ', x))
test["text"] = test["text"].apply(lambda x: re.sub(r'[^\w]', ' ', x))
test["text"] = test["text"].apply(lambda x: re.sub(r'_', ' ', x))
test["text"] = test["text"].apply(lambda x: re.sub(r'[0-9]',' ', x))
test["text"] = test["text"].apply(lambda x: re.sub(' +',' ', x))
test["text"] = test["text"].apply(lambda x: unidecode.unidecode(x))
test["text"] = test["text"].str.strip()
test["text_length"] = test["text"].str.len()

test["mean_word_length"] = test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["vowels_count"] = test["text"].apply(lambda x: count_vowels(x))
test["short_words_count"] = test["text"].apply(lambda x: count_short_words(x))
test["stopwords_count"] = test["text"].apply(lambda x: count_stopwords(x))
test["text"] = test["text"].apply(lambda x: re.sub(r'\b\w{1}\b', '', x))
test["words_count"] = test["text"].str.split(' ').apply(lambda x: len(x))

test["keyword"] = test["keyword"].str.replace('%20',' ')
test["keyword"] = test["keyword"].astype('category')

test.rename(columns={"location":"location_original"}, inplace=True)
test.rename(columns={"id":"id_original"}, inplace=True)
test.rename(columns={"text":"text_original"}, inplace=True)
test.rename(columns={"keyword":"keyword_original"}, inplace=True)

test.head()

,id_original,keyword_original,location_original,text_original,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count
0,0,NaN,NaN,just happened terrible car crash,0,,,0,0,0,0,happened terrible car crash,34,4.833333,10,2,2,6
1,2,NaN,NaN,heard about earthquake is different cities sta...,3,#earthquake,,1,0,0,0,heard earthquake different cities stay safe ev...,61,5.888889,24,1,2,9
2,3,NaN,NaN,there is forest fire at spot pond geese are f...,2,,,0,0,0,0,forest fire spot pond geese fleeing across str...,94,4.000000,31,7,9,19
3,9,NaN,NaN,apocalypse lighting spokane wildfires,3,#spokane #wildfires,,2,0,0,0,apocalypse lighting spokane wildfires,37,8.500000,12,0,0,4
4,11,NaN,NaN,typhoon soudelor kills in china and taiwan,0,,,0,0,2,0,typhoon soudelor kills china taiwan,42,5.142857,14,2,2,7


In [11]:
binary_encoder = ce.BinaryEncoder(cols=['keyword_original'])
binary_train = binary_encoder.fit_transform(train['keyword_original'])
binary_test = binary_encoder.fit_transform(test['keyword_original'])
train_bin = pd.concat([train, binary_train], axis=1)
train_bin.head()

,id_original,keyword_original,location_original,text_original,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count,keyword_original_0,keyword_original_1,keyword_original_2,keyword_original_3,keyword_original_4,keyword_original_5,keyword_original_6,keyword_original_7,keyword_original_8
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,1,#earthquake,,1,0,0,0,deeds reason earthquake may allah forgive us,68,4.307692,25,7,6,13,0,0,0,0,0,0,0,0,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1,1,,,0,0,0,0,forest fire near la ronge sask canada,37,4.428571,13,1,0,7,0,0,0,0,0,0,0,0,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,3,,,0,0,0,0,residents asked shelter place notified officer...,130,4.954545,45,9,11,22,0,0,0,0,0,0,0,0,1
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1,2,#wildfires,,1,0,5,0,people receive wildfires evacuation orders cal...,56,7.142857,24,1,1,7,0,0,0,0,0,0,0,0,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,2,#alaska #wildfires,,2,0,0,0,got sent photo ruby alaska smoke wildfires pou...,85,4.375000,25,3,7,16,0,0,0,0,0,0,0,0,1


In [12]:
test_bin = pd.concat([test, binary_test], axis=1)
test_bin.head()

,id_original,keyword_original,location_original,text_original,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count,keyword_original_0,keyword_original_1,keyword_original_2,keyword_original_3,keyword_original_4,keyword_original_5,keyword_original_6,keyword_original_7,keyword_original_8
0,0,NaN,NaN,just happened terrible car crash,0,,,0,0,0,0,happened terrible car crash,34,4.833333,10,2,2,6,0,0,0,0,0,0,0,0,1
1,2,NaN,NaN,heard about earthquake is different cities sta...,3,#earthquake,,1,0,0,0,heard earthquake different cities stay safe ev...,61,5.888889,24,1,2,9,0,0,0,0,0,0,0,0,1
2,3,NaN,NaN,there is forest fire at spot pond geese are f...,2,,,0,0,0,0,forest fire spot pond geese fleeing across str...,94,4.000000,31,7,9,19,0,0,0,0,0,0,0,0,1
3,9,NaN,NaN,apocalypse lighting spokane wildfires,3,#spokane #wildfires,,2,0,0,0,apocalypse lighting spokane wildfires,37,8.500000,12,0,0,4,0,0,0,0,0,0,0,0,1
4,11,NaN,NaN,typhoon soudelor kills in china and taiwan,0,,,0,0,2,0,typhoon soudelor kills china taiwan,42,5.142857,14,2,2,7,0,0,0,0,0,0,0,0,1


In [13]:
# feature selection
def select_features(X_train, y_train, X_test, function, k):
    fs = SelectKBest(score_func=function, k=k)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs

## XGBoost con Binary Encoding y sin BOW

In [ ]:
X = train_bin.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text", "hashtags", "labels"], axis=1)
y = train_bin["target_label"]

print(X.head().T)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

                             0          1           2          3       4
special_chars_count   1.000000   1.000000    3.000000   2.000000   2.000
hashtags_count        1.000000   0.000000    0.000000   1.000000   2.000
labels_count          0.000000   0.000000    0.000000   0.000000   0.000
num_chars_count       0.000000   0.000000    0.000000   5.000000   0.000
links_count           0.000000   0.000000    0.000000   0.000000   0.000
text_length          68.000000  37.000000  130.000000  56.000000  85.000
mean_word_length      4.307692   4.428571    4.954545   7.142857   4.375
vowels_count         25.000000  13.000000   45.000000  24.000000  25.000
short_words_count     7.000000   1.000000    9.000000   1.000000   3.000
stopwords_count       6.000000   0.000000   11.000000   1.000000   7.000
words_count          13.000000   7.000000   22.000000   7.000000  16.000
keyword_original_0    0.000000   0.000000    0.000000   0.000000   0.000
keyword_original_1    0.000000   0.000000    0.0000

In [ ]:
# GridSearch CV
# hiper_parametros = {"n_estimators":[50,100,150],
#                    "max_depth":[5,7,9],
#                    "colsample_bytree":[0.5,0.7],
#                    "subsample":[0.7,1],
#                    "learning_rate":[0.1],
#                     "n_jobs": [1]}
# XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.7, gamma=0,
#               learning_rate=0.1, max_delta_step=0, max_depth=7,
#               min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
#               nthread=None, objective='binary:logistic', random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=0.7, verbosity=1)
# 0.7031002485086537


hiper_parametros = {"n_estimators":[200,300,500],
                   "max_depth":[7,9,11,13],
                   "colsample_bytree":[0.5,0.7],
                   "subsample":[0.7,1],
                   "learning_rate":[0.1],
                    "n_jobs": [1]}

In [ ]:
clasif = GridSearchCV(xgb.XGBClassifier(), hiper_parametros, cv=4, scoring='accuracy')
clasif.fit(X_train, y_train)

GridSearchCV(cv=4, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'colsample_bytree': [0.5, 0.7], 'learning_rate': [0.1],
                         'max_depth': [7, 9, 11, 13],
                         'n_estimators': [

In [ ]:
print(clasif.best_estimator_)
print(clasif.best_score_)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
0.7008234793567041


## XGBoost con Binary Encoding y con BOW

In [14]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(train["clean_text"])

vec_train = vectorizer.fit_transform(train["clean_text"])
feature_words = vectorizer.get_feature_names()
print(len(feature_words))
df_bow_train = pd.DataFrame(vec_train.toarray(), columns=feature_words)
df_bow_train.head()

14190


,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,aashiqui,ab,aba,abandon,abandoned,abandoning,abbandoned,abbott,abbruchsimulator,abbswinston,abc,abceyewitness,abcnews,abcs,abe,aberdeen,aberystwythshrewsbury,abes,abha,abia,ability,abject,ablaze,able,ableg,aboard,abomb,abombed,abomination,abortion,...,zayn,zeal,zehrs,zenit,zergele,zero,zeros,zhejiang,zicac,zimbabwe,zimmer,zimmerman,zimpapersviews,zionism,zionist,zionists,zip,ziphimup,zipped,zippednews,zipper,zippers,zippoline,ziuw,zmne,zodiac,zombie,zombiefunrun,zombies,zone,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
vec_test = vectorizer.transform(test["clean_text"])
df_bow_test = pd.DataFrame(vec_test.toarray(), columns=feature_words)
df_bow_test.head()

,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,aashiqui,ab,aba,abandon,abandoned,abandoning,abbandoned,abbott,abbruchsimulator,abbswinston,abc,abceyewitness,abcnews,abcs,abe,aberdeen,aberystwythshrewsbury,abes,abha,abia,ability,abject,ablaze,able,ableg,aboard,abomb,abombed,abomination,abortion,...,zayn,zeal,zehrs,zenit,zergele,zero,zeros,zhejiang,zicac,zimbabwe,zimmer,zimmerman,zimpapersviews,zionism,zionist,zionists,zip,ziphimup,zipped,zippednews,zipper,zippers,zippoline,ziuw,zmne,zodiac,zombie,zombiefunrun,zombies,zone,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
filter_bow_train = df_bow_train.loc[:,(df_bow_train.sum()>1)]
filter_bow_train.head()

,aa,aba,abandon,abandoned,abbott,abbswinston,abc,abcnews,abe,abia,ability,ablaze,able,abomb,abomination,abortion,absolute,absolutely,abstorm,abuse,abused,abusing,ac,acc,accept,access,accident,accidentally,accidents,according,account,accounts,accused,accuses,accustomed,acdelco,achimota,acid,acoustic,acquire,...,years,yelling,yellow,yemen,yep,yes,yesterday,yesterdays,yo,yobe,yonews,york,yorker,youd,youll,young,younger,youre,youth,youtube,youve,ypres,yr,yrs,yugvani,yyc,yycstorm,zabadani,zarry,zayn,zero,zimbabwe,zionism,zionist,zippednews,zombie,zone,zones,zouma,zss
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
filter_list = filter_bow_train.columns.tolist()
filter_bow_test = df_bow_test.filter(items=filter_list)
filter_bow_test.shape

(3263, 6049)

In [18]:
train_bow_bin = pd.concat([train_bin,filter_bow_train], axis="columns")
print(train_bow_bin.shape)

(7613, 6077)


In [19]:
train_bow_bin.sample(5)

,id_original,keyword_original,location_original,text_original,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count,keyword_original_0,keyword_original_1,keyword_original_2,keyword_original_3,keyword_original_4,keyword_original_5,keyword_original_6,keyword_original_7,keyword_original_8,aa,aba,abandon,abandoned,abbott,abbswinston,abc,abcnews,abe,abia,ability,ablaze,...,years,yelling,yellow,yemen,yep,yes,yesterday,yesterdays,yo,yobe,yonews,york,yorker,youd,youll,young,younger,youre,youth,youtube,youve,ypres,yr,yrs,yugvani,yyc,yycstorm,zabadani,zarry,zayn,zero,zimbabwe,zionism,zionist,zippednews,zombie,zone,zones,zouma,zss
403,580,arson,NaN,mourning notices for stabbing arson victims st...,1,11,,,0,0,7,1,mourning notices stabbing arson victims stir u...,81,4.857143,27,5,3,14,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2296,3295,demolish,KOLKATA,jolly jinu you said they are terrorist because...,0,5,#babri,@jolly_jinu,1,1,1,0,said terrorist babri ok demolish house todayha...,127,4.120000,45,12,13,25,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1385,1996,bush fires,Loughborough.,johnfromcranber pleas for global warming don ...,1,6,,@johnfromcranber,0,1,0,0,pleas global warming dont really work californ...,120,6.117647,37,3,5,17,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7235,10361,weapons,Multinational *****,jamesmelville some old testimony of weapons us...,0,11,,@jamesmelville,0,1,2,1,old testimony weapons used promote conflicts t...,115,6.250000,38,5,4,16,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2592,3721,destroyed,NaN,russian customs destroyed total of tons of fo...,0,2,,,0,0,3,0,russian customs destroyed total tons food toda...,128,4.863636,45,6,8,22,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
test_bow_bin = pd.concat([test_bin,filter_bow_test], axis="columns")
print(test_bow_bin.shape)
test_bow_bin.sample(5)

(3263, 6076)


,id_original,keyword_original,location_original,text_original,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count,keyword_original_0,keyword_original_1,keyword_original_2,keyword_original_3,keyword_original_4,keyword_original_5,keyword_original_6,keyword_original_7,keyword_original_8,aa,aba,abandon,abandoned,abbott,abbswinston,abc,abcnews,abe,abia,ability,ablaze,able,...,years,yelling,yellow,yemen,yep,yes,yesterday,yesterdays,yo,yobe,yonews,york,yorker,youd,youll,young,younger,youre,youth,youtube,youve,ypres,yr,yrs,yugvani,yyc,yycstorm,zabadani,zarry,zayn,zero,zimbabwe,zionism,zionist,zippednews,zombie,zone,zones,zouma,zss
1112,3673,destroy,NaN,but bad people kiss too hehe their kisses can ...,3,,,0,0,0,0,bad people kiss hehe kisses destroy,53,4.400000,17,4,4,10,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1663,5607,flood,New York,cree led work light bar alloy spot flood comb...,15,,,0,0,10,2,w cree led work light bar alloy spot flood com...,83,3.941176,24,5,0,17,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
829,2721,crushed,NaN,so over getting my hopes up and then being c...,2,,,0,0,0,0,im getting hopes crushed,54,3.583333,15,6,9,12,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
724,2355,collapsed,"Cramerton, NC",that ppact hasn already collapsed is testame...,4,,@ppact @thedemocrats,0,2,0,0,hasnt already collapsed testament lockstep dev...,109,5.470588,34,5,8,17,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
568,1854,burned,"Ecruteak City, Johto",alfred dle pdleague sorry about that crit no...,4,,@alfred00dle @pdleague,0,2,5,0,im sorry crit sure matter prob cuz burned,89,3.736842,27,9,9,19,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
X = train_bow_bin.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text", "hashtags", "labels"], axis=1)
y = train_bow_bin["target_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 6069)
(5709,)
(1904, 6069)
(1904,)


In [22]:
# utilizando las 6069 features
model = xgb.XGBClassifier(n_estimators=500, max_depth=11, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.806197


In [ ]:
#me quedo con las mejores
X_train_fs, X_test_fs = select_features(X_train, y_train, X_test, chi2, 1000)
X_train_fs2, X_test_fs2 = select_features(X_train, y_train, X_test, chi2, 2000)

In [ ]:
model = xgb.XGBClassifier(n_estimators=500, max_depth=7, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.795168


In [ ]:
model = xgb.XGBClassifier(n_estimators=400, max_depth=11, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.804097


In [ ]:
# me quedo con 3000 features
X_train_fs, X_test_fs = select_features(X_train, y_train, X_test, chi2, 3000)

In [ ]:
model = xgb.XGBClassifier(n_estimators=400, max_depth=11, colsample_bytree=0.7, learning_rate=0.1, subsample=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.798845


In [ ]:
vectorizer = CountVectorizer(stop_words='english', binary=True)
vectorizer.fit(train["clean_text"])

vec_train = vectorizer.fit_transform(train["clean_text"])
feature_words = vectorizer.get_feature_names()
print(len(feature_words))
df_bow_train = pd.DataFrame(vec_train.toarray(), columns=feature_words)
df_bow_train.head()

14190


,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,aashiqui,ab,aba,abandon,abandoned,abandoning,abbandoned,abbott,abbruchsimulator,abbswinston,abc,abceyewitness,abcnews,abcs,abe,aberdeen,aberystwythshrewsbury,abes,abha,abia,ability,abject,ablaze,able,ableg,aboard,abomb,abombed,abomination,abortion,...,zayn,zeal,zehrs,zenit,zergele,zero,zeros,zhejiang,zicac,zimbabwe,zimmer,zimmerman,zimpapersviews,zionism,zionist,zionists,zip,ziphimup,zipped,zippednews,zipper,zippers,zippoline,ziuw,zmne,zodiac,zombie,zombiefunrun,zombies,zone,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_bow_bin = pd.concat([train_bin,df_bow_train], axis="columns")
print(train_bow_bin.shape)

(7613, 14218)


In [ ]:
X = train_bow_bin.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text", "hashtags", "labels"], axis=1)
y = train_bow_bin["target_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 14209)
(5709,)
(1904, 14209)
(1904,)


In [ ]:
#me quedo con las mejores
X_train_fs, X_test_fs = select_features(X_train, y_train, X_test, chi2, 700)
X_train_fs2, X_test_fs2 = select_features(X_train, y_train, X_test, chi2, 1000)
X_train_fs3, X_test_fs3 = select_features(X_train, y_train, X_test, chi2, 2000)

In [ ]:
model = xgb.XGBClassifier(n_estimators=350, max_depth=7, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=350, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.792017


In [ ]:
model = xgb.XGBClassifier(n_estimators=500, max_depth=9, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.794118


In [ ]:
model = xgb.XGBClassifier(n_estimators=500, max_depth=13, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train_fs3, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=13,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs3)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.786765


In [ ]:
# reduciendo las features

In [ ]:
# utilizando las 6069 features
model = xgb.XGBClassifier(n_estimators=500, max_depth=11, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train, y_train)

In [23]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.784664


In [24]:
df_feat_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)
df_feat_importances.head()

,importancia
malaysia,0.013926
fatal,0.012855
body,0.009965
northern,0.006683
killed,0.006614


In [26]:
list_fi = df_feat_importances.index[:2000].tolist()
X_fi = X.filter(items=list_fi)
print(X_fi.shape)
print(X_fi.columns)

(7613, 2000)
Index(['malaysia', 'fatal', 'body', 'northern', 'killed', 'pm', 'desolation',
       'suicide', 'california', 'dead',
       ...
       'targeting', 'targets', 'tasmanias', 'tb', 'tdp', 'tcot', 'waking',
       'tch', 'tc', 'walerga'],
      dtype='object', length=2000)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_fi, y, test_size=0.25, random_state=100)

In [28]:
model = xgb.XGBClassifier(n_estimators=500, max_depth=11, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [29]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.808298


In [31]:
X_train = train_bow_bin.drop(["id_original","keyword_original","location_original","text_original","target_label","hashtags","labels","clean_text"], axis=1)
X_test = test_bow_bin.drop(["id_original","keyword_original","location_original","text_original","hashtags","labels","clean_text"], axis=1)
y_train = train_bow_bin["target_label"]

In [32]:
list_fi = df_feat_importances.index[:2000].tolist()
X_train_fi = X_train.filter(items=list_fi)
X_test_fi = X_test.filter(items=list_fi)

In [33]:
print(X_train_fi.shape)
print(X_test_fi.shape)

(7613, 2000)
(3263, 2000)


In [34]:
model = xgb.XGBClassifier(n_estimators=500, max_depth=11, colsample_bytree=0.5, learning_rate=0.1, subsample=1)
model.fit(X_train_fi, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [35]:
y_pred = model.predict(X_test_fi)
y_pred

array([0, 1, 1, ..., 1, 1, 0])

In [36]:
test["target"] = y_pred
test[["id_original","target"]]

,id_original,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [37]:
from google.colab import files

test[["id_original","target"]].rename(columns={"id_original":"id"}).to_csv("pred9_XGB_bowbin", index=False)
files.download("pred9_XGB_bowbin")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>